Условия

---

$i = 4$

$i = 5$

$\lambda = \frac{1+i}{4} = \frac{5}{4} - \text{среднее кол-во сообщений в минуту}$

$n = 3+[\frac{i+j}{8}] = 4 - \text{кол-во каналов}$ 

$\tau = \frac{5}{5+j} = 0.5 - \text{среднее время обработки сообщения}$



In [37]:
from math import factorial

lbd = 5/4 
tau = 0.5

mu = 1/tau
rho = lbd/mu

ch_n = 5

p_0 = 0 
for i in range(0,ch_n):
    p_0 += rho**i/factorial(i)

p_0 = p_0 ** -1

print('p_0:',p_0)

p_arr = [p_0]

for i in range(1,ch_n):
    p_i = (rho**i/factorial(i))*p_0
    print('p_'+str(i)+':',p_i)
    p_arr.append(p_i)

Q = 1 - p_arr[-1]
A = lbd*Q

print('\nТеоретические значения.')
print('Абсолютная пропускная способность: ',A)
print('Относительная пропускная способность: ',Q)
print('Вероятность отказа: ',p_arr[-1])

p_0: 0.5355152558438516
p_1: 0.33469703490240726
p_2: 0.10459282340700227
p_3: 0.021790171543125472
p_4: 0.0034047143036133555

Теоретические значения.
Абсолютная пропускная способность:  1.2457441071204833
Относительная пропускная способность:  0.9965952856963867
Вероятность отказа:  0.0034047143036133555


In [38]:
import numpy 
import cProfile
from tqdm import tqdm
delta_t = 0.001
t_meansurment_val = (delta_t**-1)
class Channel:
    def __init__(self):
        self.work_timer = 0
        self.state = True
    
    def is_empty(self):
        return self.state
    
    def set_work(self,work_time):
        if self.state:
            self.state = False
            self.work_timer = work_time
    
    def step(self):
        if not self.state and self.work_timer:
            self.work_timer -= delta_t
            if self.work_timer < delta_t:
                self.state = True
                self.work_timer = 0

class Network:
    def __init__(self,ch_count,intensivity,avr_compute_time):
        self.intensivity = intensivity
        self.avr_compute_time = avr_compute_time
        self.rnd_gen = numpy.random.default_rng()
        self.next_msg = round(self.rnd_gen.poisson(self.intensivity),3)
        self.total_time = 0
        self.total_task = 0
        self.miss_task = 0
        self.channels = [] 
        self.task_times = []
        self.empty_channels = []
        for i in range(0,ch_count):
            self.channels.append(Channel())

### Возвращает номер свободного канала, если все каналы заняты возвращает -1
    def check_free_channel(self):
        for index,i in enumerate(self.channels):
            if i.is_empty():
                return index 
        return -1

    def empty_channels_count(self):
        state_count = 0
        for i in self.channels:
            state_count += int(i.is_empty())
        return state_count

    def step(self):
        if self.next_msg < delta_t:
            self.new_task()
        else:
            self.next_msg -= delta_t

        for i in self.channels:
            i.step()

        self.total_time += delta_t

        self.empty_channels.append(self.empty_channels_count())

    def new_task(self):
        self.next_msg = round(self.rnd_gen.poisson(self.intensivity),3)
        free_channel = self.check_free_channel()
        if not (free_channel == -1):
            self.total_task +=1
            task_time = round(self.rnd_gen.normal(self.avr_compute_time,0.1),3)
            self.task_times.append(task_time)
            self.channels[free_channel].set_work(task_time)
        else:
            self.miss_task += 1

    def get_avr_task_time(self):
        return numpy.mean(self.task_times)
    
    def get_avr_empty_chanells(self):
        return numpy.mean(self.empty_channels)
    
    def get_total_tasks(self):
        return self.total_task

    def get_missed_tasks(self):
        return self.miss_task

n = t_meansurment_val*60*24
net = Network(4,5/4,0.5)
for i in range(0,int(n)):
    net.step()

print('\nПрактические значения.')
print('Обработано сообщений: ',net.get_total_tasks())
print('Отказано в обработке: ',net.get_missed_tasks())
print('Среднее время обработки: ',net.get_avr_task_time())
print('Среднее кол-во занятых каналов: ',4-net.get_avr_empty_chanells())
print('Абсолютная пропускная способность: ',t_meansurment_val*(net.get_total_tasks()-net.get_missed_tasks())/n)
print('Относительная пропускная способность: ',(net.get_total_tasks()-net.get_missed_tasks())/net.get_total_tasks())
print('Вероятность отказа: ',1-(net.get_total_tasks()-net.get_missed_tasks())/net.get_total_tasks())


Практические значения.
Обработано сообщений:  1123
Отказано в обработке:  10
Среднее время обработки:  0.5017916295636687
Среднее кол-во занятых каналов:  0.3912497222222222
Абсолютная пропускная способность:  0.7729166666666667
Относительная пропускная способность:  0.9910952804986642
Вероятность отказа:  0.008904719501335756
